In [1]:
!pip install selenium

In [40]:
from selenium import webdriver
import os
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd
import datetime
import re

## Construimos las url´s necesarias para hacer el scrapping. Se le agrega la fecha de todos los días del año al objeto "URL"

In [3]:
url='https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/'
lista_url=[]

for i in pd.Series(pd.date_range(start='2015-01-01', end='2018-05-31')).astype(str):
       lista_url.append(url+str(i))

## Comenzamos con el scrapeo.....

In [5]:
#Creamos un dataframe vacio y un contador

contador=0

df_empty=pd.DataFrame()

#Creamos lista vacia para incorporar las url de donde se extrae cada fila de información. 
#Luego va a ser útil para captar la fecha a la que corresponde cada fila.

lista_fechas=[]

## Seteamos las opciones del driver para que inicie el navegador expandiéndolo

option = webdriver.ChromeOptions()

option.add_argument('start-maximized')

option.add_argument('incognito')

## abro el navegador
current_location = os.getcwd()

browser = webdriver.Chrome(executable_path=current_location+'/chromedriver', options=option)

sleep(2)

##indico la web desde dónde empezar

browser.get(lista_url[0])

sleep(3)

## cambio el formato de las unidades de medida (de F a Celcius)

browser.find_elements_by_xpath('//*[@id="wuSettings"]/i')[0].click()

sleep(1)

browser.find_elements_by_xpath('//*[@id="wuSettings-quick"]/div/a[2]')[0].click()

## for loop para buscar todas las URL de la lista y extraer todas las filas de la tabla del clima 
## Algunas tablas tienen registros con frecuencia menor a una hora. 

for url in lista_url:
    
    contador+=1
    
    print(contador)
    
    print(url)
    
    try:
        
        browser.get(url)
    
        sleep(2)
        
    except:
        
        browser.close()
        
    for i in range(1,25,1):
        
        try:
            
            df_empty=df_empty.append([browser.find_elements_by_xpath('//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(i)+']')[0].text.split(' ')])
            
            lista_fechas.append(browser.current_url)
            
        except: 
            
            print('hay menos de 30 registros en ese día')        
    

1
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-01
2
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-02
3
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-03
4
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-04
5
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-05
6
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-06
7
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-07
8
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-08
9
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-09
10
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-10
11
https://www.wunderground.com/history/daily/ar/buenos-aires/SABE/date/2015-01-11
hay menos de 30 registros en ese día
12
https://www.wunderground.com/history/daily/ar/buenos-air

In [7]:
df_empty.to_csv('data_clima.csv')

In [17]:
## Eliminamos las columnas que no vamos a necesitar
df_empty_clean = df_empty.drop([2,3,4,5,6,7,8,10,11,12,13,14,16,21],axis = 1) 

In [18]:
## Concatenamos columnas para dejar una variable limpia de "Condition"
df_empty_clean['Condition'] = df_empty_clean[17].map(str) + ' ' + df_empty_clean[18].map(str) + ' ' + df_empty_clean[19].map(str)+ ' ' + df_empty_clean[20].map(str)
df_empty_clean_2 = df_empty_clean.drop([17,18,19,20],axis = 1)

In [19]:
#Limpiamos los valores que componen "Condition" para normalizarlos
df_empty_clean_2['Condition'] = df_empty_clean_2['Condition'].replace('nan','',regex=True).str.strip()
df_empty_clean_2.reset_index(drop=True, inplace=True)

In [20]:
df_empty_clean_2

,0,1,9,15,Condition
0,12:00,AM,2,0.0,Fair
1,1:00,AM,2,0.0,Fair
2,2:00,AM,7,0.0,Mostly Cloudy
3,3:00,AM,9,0.0,Fair
4,4:00,AM,7,0.0,Fair
...,...,...,...,...,...
15480,7:00,PM,9,0.0,Light Drizzle
15481,8:00,PM,13,0.0,Light Drizzle
15482,9:00,PM,9,0.0,Light Drizzle
15483,10:00,PM,13,0.0,Light Drizzle


In [21]:
#Renombramos las columnas del dataframe
df_empty_clean_2.columns=["Hour","AM/PM","Wind_Speed","Rain_Ml","Condition"]

In [22]:
df_empty_clean_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15485 entries, 0 to 15484
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Hour        15485 non-null  object
 1   AM/PM       15485 non-null  object
 2   Wind_Speed  15485 non-null  object
 3   Rain_Ml     15485 non-null  object
 4   Condition   15485 non-null  object
dtypes: object(5)
memory usage: 605.0+ KB


In [23]:
#Creamos una serie con todas las fechas que obtuvimos durante el scrapping (Fecha de inicio y Fecha de finalización)
fecha=pd.Series(lista_fechas).str.extract('(\d+-\d+-\d+)')
fecha.reset_index(drop=True, inplace=True)

In [121]:
#Concatenamos la serie fecha al dataframe de clima para dejar uno solo consolidado
frames = [fecha, df_empty_clean_2]
df_empty_final = pd.concat(frames,axis = 1)
#Cambiamos el nombre de la columna correspondiente a Fecha
df_empty_final.rename(columns={0:'Date'}, inplace=True)

In [122]:
#Eliminamos registros no numéricos
                      
df_empty_final.drop(df_empty_final.index[df_empty_final['Wind_Speed'] == 'km/h'], inplace = True)
df_empty_final.drop(df_empty_final.index[df_empty_final['Rain_Ml'] == 'mm'], inplace = True)

In [123]:
#Alteramos el formato de algunas variables                   
df_empty_final['Date'] = pd.to_datetime(df_empty_final['Date'])
df_empty_final['Rain_Ml'] = pd.to_numeric(df_empty_final['Rain_Ml'])
df_empty_final['Wind_Speed'] = pd.to_numeric(df_empty_final['Wind_Speed'].apply(lambda x: re.sub(',', '.', x)))

In [124]:

df_empty_final['Only_Hour'] = df_empty_final.Hour.str.split(":")
df_empty_final['Only_Hour']  = [x[0] for x in list(df_empty_final['Only_Hour'])]


In [125]:
df_empty_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15472 entries, 0 to 15484
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        15472 non-null  datetime64[ns]
 1   Hour        15472 non-null  object        
 2   AM/PM       15472 non-null  object        
 3   Wind_Speed  15472 non-null  float64       
 4   Rain_Ml     15472 non-null  float64       
 5   Condition   15472 non-null  object        
 6   Only_Hour   15472 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1.6+ MB


In [126]:
df_empty_final

,Date,Hour,AM/PM,Wind_Speed,Rain_Ml,Condition,Only_Hour
0,2015-01-01,12:00,AM,2.0,0.0,Fair,12
1,2015-01-01,1:00,AM,2.0,0.0,Fair,1
2,2015-01-01,2:00,AM,7.0,0.0,Mostly Cloudy,2
3,2015-01-01,3:00,AM,9.0,0.0,Fair,3
4,2015-01-01,4:00,AM,7.0,0.0,Fair,4
...,...,...,...,...,...,...,...
15480,2018-05-31,7:00,PM,9.0,0.0,Light Drizzle,7
15481,2018-05-31,8:00,PM,13.0,0.0,Light Drizzle,8
15482,2018-05-31,9:00,PM,9.0,0.0,Light Drizzle,9
15483,2018-05-31,10:00,PM,13.0,0.0,Light Drizzle,10


In [127]:
#Guardamos un csv con el Df consolidado
df_empty_final.sort_values(by=['Date','AM/PM','Hour']).to_csv('clima.csv', index=False)

In [128]:
df_empty_final.reset_index(drop=True, inplace=True)

In [130]:
n=0
for index, row in df_empty_final.iterrows():
    
    if (row['AM/PM'] == 'PM' and row['Only_Hour'] != "12") or (row['AM/PM'] == 'AM' and row['Only_Hour'] == "12"):
        full_hour = di[row['Only_Hour']]
    else:    
        full_hour = row['Only_Hour']
        
    print(full_hour, n)
        
    df_empty_final.loc[n, 'Full_Hour'] = full_hour

    n+=1

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9
10 10
10 11
11 12
12 13
13 14
14 15
15 16
16 17
17 18
18 19
19 20
20 21
21 22
22 23
0 24
1 25
2 26
3 27
4 28
5 29
6 30
7 31
8 32
9 33
10 34
11 35
12 36
13 37
14 38
15 39
16 40
17 41
18 42
19 43
20 44
21 45
22 46
23 47
0 48
1 49
2 50
3 51
4 52
5 53
6 54
7 55
8 56
9 57
10 58
11 59
12 60
13 61
14 62
15 63
16 64
17 65
18 66
18 67
19 68
20 69
21 70
22 71
0 72
0 73
1 74
2 75
3 76
4 77
5 78
6 79
7 80
8 81
9 82
10 83
11 84
12 85
13 86
14 87
15 88
16 89
17 90
18 91
19 92
20 93
21 94
22 95
0 96
1 97
2 98
3 99
4 100
5 101
6 102
7 103
8 104
9 105
10 106
11 107
12 108
13 109
14 110
15 111
16 112
17 113
18 114
19 115
20 116
21 117
22 118
23 119
0 120
1 121
2 122
3 123
4 124
5 125
5 126
6 127
6 128
7 129
8 130
9 131
10 132
11 133
12 134
12 135
13 136
14 137
15 138
16 139
17 140
18 141
19 142
20 143
0 144
1 145
2 146
3 147
4 148
5 149
6 150
7 151
8 152
9 153
10 154
11 155
12 156
13 157
14 158
15 159
16 160
17 161
18 162
19 163
20 164
21 165
22 166
23 167
0 168
1

In [148]:
df_empty_final[df_empty_final['Date']=='2015-09-07']

,Date,Hour,AM/PM,Wind_Speed,Rain_Ml,Condition,Only_Hour,Full_Hour


In [136]:
df_empty_final.to_csv('info_clima.csv')